In [1]:
import os
os.chdir("/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/Data/Visium/raw/T-GRAASP/")
import torch
from models import STx_ARGVA, STx_encoder, STx_discriminator
from utils import *
from train import Trainer

# sample_ids = ['CAQSC','WYYBT','XXGBT','ZWFBT','ZWJBT']
sample_ids = ['Young1','Mid1','Old1','Young2', 'Mid2',  'Old2']
# sample_ids = ['Young1']
raw_dir = '/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/Data/Visium/raw'
test_ratio = 0.1


sp_graph_list, sp_adj_list = SGEDataset(sample_ids, raw_dir, test_ratio=0.10)
# sc_graph_list = SGEDataset1(sample_ids, raw_dir)

N_nodes   = sp_graph_list[0].x.size(0)   # 节点个数
F_inputs  = sp_graph_list[0].x.size(1)   

ppi_file     = f'{raw_dir}/PPI1.connect.txt'
connectivity = load_ppi_connectivity(ppi_file, F_inputs)    # shape = [2, E]




Loading sample Young1 
Loading sample Mid1 
Loading sample Old1 
Loading sample Young2 
Loading sample Mid2 
Loading sample Old2 


In [2]:
encoder = STx_encoder(
    in_channels      = F_inputs,        
    hidden_channels  = 96,
    out_channels     = 16,
    m = 29, l = 3, K = 3,
    connect = connectivity,        
    pi = 0.75,
    n_heads = 8,
    activate_sc_alignment=False
)

discriminator = STx_discriminator(
    in_channels     = 16,  
    hidden_channels = 8,   
    out_channels    = 1
)

model = STx_ARGVA(
    encoder      = encoder,
    discriminator= discriminator,
    l            = 3          
)

In [3]:
from torch.utils.data import DataLoader
train_loader = DataLoader(sp_graph_list, batch_size=1, shuffle=True)

In [19]:
sp_graph_list

[Data(x=[2573, 2673], y=[2573], train_edge_index=[2, 9042], test_edge_index=[2, 1004], pos_edge=[10046, 2], neg_edge=[2, 1004], train_edge_weight=[9042], test_edge_weight=[1004], batch=[2573]),
 Data(x=[2735, 2673], y=[2735], train_edge_index=[2, 9630], test_edge_index=[2, 1069], pos_edge=[10699, 2], neg_edge=[2, 1069], train_edge_weight=[9630], test_edge_weight=[1069], batch=[2735]),
 Data(x=[2973, 2673], y=[2973], train_edge_index=[2, 10495], test_edge_index=[2, 1166], pos_edge=[11661, 2], neg_edge=[2, 1166], train_edge_weight=[10495], test_edge_weight=[1166], batch=[2973]),
 Data(x=[2804, 2673], y=[2804], train_edge_index=[2, 9883], test_edge_index=[2, 1098], pos_edge=[10981, 2], neg_edge=[2, 1098], train_edge_weight=[9883], test_edge_weight=[1098], batch=[2804]),
 Data(x=[2801, 2673], y=[2801], train_edge_index=[2, 9874], test_edge_index=[2, 1097], pos_edge=[10971, 2], neg_edge=[2, 1097], train_edge_weight=[9874], test_edge_weight=[1097], batch=[2801]),
 Data(x=[2857, 2673], y=[285

In [5]:
import os
os.chdir("/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/Data/Visium/raw/T-GRAASP/")
from trainer import MultiDataTrainer
pretrain_lists = [[d] for d in sp_graph_list[0:3]]

multi_trainer = MultiDataTrainer(
    model=model,
    sp_graph_lists=pretrain_lists,
    sc_graph_lists=None,   # 若不用可直接传 []
    pretrain_epochs=4,
    patience=10,
    save_dir='./checkpoints_multi'
)

best_model_state = multi_trainer.pretrain()
# print("整体AUC均值最优的一份state_dict:", best_model_state)

=== Data Structure Debug Info ===
Number of datasets: 3
Start Pretrain...
[Pretrain] Epoch 001 | Dataset 0 | Loss 18.4469 | ARI 0.245 | ACC 0.398 | NMI 0.464 | AUC 0.553 | AP 0.580 | ACC1 0.502
[Pretrain] Epoch 001 | Dataset 1 | Loss 18.6884 | ARI 0.194 | ACC 0.351 | NMI 0.441 | AUC 0.659 | AP 0.668 | ACC1 0.542
[Pretrain] Epoch 001 | Dataset 2 | Loss 18.6475 | ARI 0.192 | ACC 0.351 | NMI 0.424 | AUC 0.638 | AP 0.654 | ACC1 0.574
The best mean AUROC is 0.6165, Epoch 1）
Epoch 1
Average Loss across all datasets: 18.5943
Mean AUC across all datasets: 0.6165 | Best Mean: 0.6165 (Epoch 1)
[Pretrain] Epoch 002 | Dataset 0 | Loss 25.4324 | ARI 0.247 | ACC 0.400 | NMI 0.475 | AUC 0.624 | AP 0.645 | ACC1 0.561
[Pretrain] Epoch 002 | Dataset 1 | Loss 18.4577 | ARI 0.196 | ACC 0.354 | NMI 0.441 | AUC 0.666 | AP 0.675 | ACC1 0.575
[Pretrain] Epoch 002 | Dataset 2 | Loss 18.4079 | ARI 0.195 | ACC 0.357 | NMI 0.429 | AUC 0.684 | AP 0.651 | ACC1 0.511
The best mean AUROC is 0.6579, Epoch 2）
Epoch 2
A

In [ ]:
model.load_state_dict(torch.load('/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/GBM/Verify/initial_data/result/best_model_knn_y1.pkl'))

trainer = Trainer(
    device="cuda:1",
    model=model,
    sp_graph_list=sp_graph_list[2:3],
    sc_graph_list=sc_graph_list[2:3],
    sp_adj_list=sp_adj_list[2:3]
    )

best_model_state = trainer.finetune(patience=1000, lr=1e-3, epochs=1500)
torch.save(best_model_state,'/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/GBM/Verify/model/finetuned_best_model_XXG.pkl')